In [4]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import glob
import dask
import datetime
from dateutil.relativedelta import relativedelta
from functools import partial
import pandas as pd
import sys
from CASutils import readdata_utils as read

from smyleutils import filter_utils as filt

dask.config.set(**{'array.slicing.split_large_chunks': True})

In [5]:
ystart = 1970 # start year of hindcasts 
yend = 2020 # end year of hindcasts
nyears = yend - ystart + 1
initmon=11
initmonstr = str(initmon).zfill(2)
nmems = 20 # the number of hindcast members
memstr = [str(i).zfill(3) for i in np.arange(1,nmems+1,1)+100] # generating member strings
topdir = "/glade/campaign/cesm/development/espwg/SMYLE-CW3E-L83/timeseries/daily/FLUT/"
expname = "b.e21.BSMYLE-CW3E-L83"
outpath="/glade/campaign/cgd/cas/islas/python/smyle_cw3e/DATA_SORT/mjo_filter/"

### Set up the dask cluster

In [6]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '10GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=10GB',
    project='P04010022',
    walltime='03:00:00',
    interface='ext')

# scale up
cluster.scale(20)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [47]:
#cluster.close()

In [8]:
cluster

PBSCluster(13b31f29, 'tcp://128.117.208.112:39419', workers=12, threads=12, memory=111.72 GiB)

### Loop over members to calculate the MJO filtered OLR for each year.  So treating each individual member like the OBS.  Removing a seasonal cycle that's just the average over years of a given day.  I would normally remove some number of harmonics but we only have 6 months in the L83.

In [9]:
def preprocessor(ds):
    # sort out the times so that each member has the same time axis
    #timebndavg = np.array(ds.time_bnds,
    #                     dtype = 'datetime64[s]').view('i8').mean(axis=1).astype('datetime64[s]')
    #ds['time'] = timebndavg
    ds = read.fixcesmtime_daily(ds)
    ds = ds.where( ds.time.dt.hour == 12, drop=True)
    datestart=pd.to_datetime("1970-"+str(ds.isel(time=0).time.dt.month.values).zfill(2)+"-"+str(ds.isel(time=0).time.dt.day.values).zfill(2)+
                       "-"+str(ds.isel(time=0).time.dt.hour.values), format="%Y-%m-%d-%H")
    time = [ datestart + relativedelta(days=int(i)) for i in np.arange(0,ds.time.size,1) ]
    ds['time'] = time
    ds = ds.sel(lat=slice(-40,40))
    return ds

In [10]:
mjostdout = []
for imem in np.arange(1,len(memstr)+1,1):
    print(imem)
    files = [glob.glob(topdir+expname+"*."+str(imem).zfill(3)+".cam.h1.FLUT."+str(iyear)+initmonstr+"01-*.nc")[0] for iyear in np.arange(ystart,yend+1,1)]
    dat = xr.open_mfdataset(files, combine='nested', concat_dim=['init_year'],
                            parallel=True, data_vars=['FLUT'], coords='minimal', compat='override', preprocess=partial(preprocessor)).FLUT
    datclim = dat.mean('time')
    sys.exit()
    datanoms = dat - datclim
    datanoms = datanoms.sel(time=slice("1970-11-01","1971-04-30"))
    
    mjofilt=[]
    for iyear in np.arange(0,datanoms.init_year.size,1):
        mjofilt.append(filt.wkfilter(datanoms.isel(init_year=iyear), 0.15, 1, 5, 20, 100, spd=1))
    mjofilt = xr.concat(mjofilt, dim='init_year')
    mjofilt = mjofilt.sel(time=slice("1970-12-01","1971-02-28"))
    mjostd = mjofilt.std('time')
    
    mjostdout.append(mjostd)
    
mjostdout = xr.concat(mjostdout, dim='M')   
mjostdout = mjostdout.rename('MJO_OLR')
mjostdout.to_netcdf(outpath+'MJOfilteredOLR_SMYLE_L83_init'+initmonstr+'.nc')
cluster.close()

1


SystemExit: 

/glade/u/home/islas/miniconda3/envs/islaenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3560: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
print(dat)

<xarray.DataArray 'FLUT' (init_year: 51, time: 181, lat: 84, lon: 288)>
dask.array<concatenate, shape=(51, 181, 84, 288), dtype=float32, chunksize=(1, 181, 84, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -39.11 -38.17 -37.23 -36.28 ... 37.23 38.17 39.11
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 1970-11-01T12:00:00 ... 1971-04-30T12:00:00
Dimensions without coordinates: init_year
Attributes:
    Sampling_Sequence:  rad_lwsw
    units:              W/m2
    long_name:          Upwelling longwave flux at top of model
    cell_methods:       time: mean


In [15]:
print(datanoms.isel(init_year=1).sel(lon=120, lat=-10, method='nearest').values)

[  41.589905     55.3692       61.60678      63.342804     57.71872
   55.29901      53.780396    -28.833298    -34.93828      31.13916
    6.153763    -36.884766    -55.060318     -1.5422668   -71.289246
  -16.874619    -34.836243     -1.4240265   -90.52904     -24.448242
   39.88745      49.09381      48.374847     42.091187     44.54431
    5.6077576    -9.080093     37.544006     42.74414      48.376526
   48.205017     48.99765      61.994476     66.420135     63.506622
   67.08823      68.45355      66.871124     62.74292      54.685455
   49.81198      55.3161       33.146606     38.341187     41.4245
   39.1485       44.055145     39.74466      31.130432     48.384186
  -12.860733     13.566345    -19.174759   -102.53122     -81.65234
  -32.383057    -65.050186    -82.45181     -46.064102    -40.436035
  -66.22        -44.47377      -4.853424     37.795197     12.044556
   25.666962     16.634552     17.224976     33.83557      38.048004
   10.350418    -58.104324    -57.613007